In [2]:
import tensorflow as tf
import numpy as np
import random
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

from tensorflow.keras.applications import (
    VGG16, VGG19, ResNet50, InceptionV3, Xception,
    MobileNet, DenseNet121, EfficientNetB1,
    NASNetMobile, NASNetLarge
)
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_pre
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_pre
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_pre
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_pre
from tensorflow.keras.applications.xception import preprocess_input as xception_pre
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_pre
from tensorflow.keras.applications.densenet import preprocess_input as densenet_pre
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_pre
from tensorflow.keras.applications.nasnet import preprocess_input as nasnet_pre

#======================= Load CIFAR-100 and prepare 20-class subset =========================#
(trainX, trainY), (testX, testY) = tf.keras.datasets.cifar100.load_data(label_mode='fine')
trainY = trainY.flatten()
testY = testY.flatten()

selected_classes = random.sample(range(100), 20)
class_map = {original: new for new, original in enumerate(selected_classes)}

test_mask = np.isin(testY, selected_classes)
testX, testY = testX[test_mask], testY[test_mask]
testY = np.vectorize(class_map.get)(testY)

testX_resized = tf.image.resize(testX, (224, 224)).numpy()

#======================= Pretrained model config ===========================================#
models_config = {
    "VGG16": (VGG16, vgg16_pre),
    "VGG19": (VGG19, vgg19_pre),
    "ResNet50": (ResNet50, resnet_pre),
    "InceptionV3": (InceptionV3, inception_pre),
    "Xception": (Xception, xception_pre),
    "MobileNet": (MobileNet, mobilenet_pre),
    "DenseNet121": (DenseNet121, densenet_pre),
    "EfficientNetB1": (EfficientNetB1, efficientnet_pre),
    "NASNetMobile": (NASNetMobile, nasnet_pre),
    "NASNetLarge": (NASNetLarge, nasnet_pre)
}

#======================= Activations to evaluate ===========================================#
activations = ["relu", "softmax", "tanh", "sigmoid"]
results = {}  #{model: {activation: accuracy}}

#======================= Evaluation Loop ===================================================#
for model_name, (ModelClass, preprocess_fn) in models_config.items():
    print(f"\n================= {model_name} =================")
    results[model_name] = {}

    testX_preprocessed = preprocess_fn(testX_resized.copy())
    base_model = ModelClass(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False

    for activation in activations:
        print(f"--- Activation: {activation}")
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        output = Dense(20, activation=activation)(x)

        model = Model(inputs=base_model.input, outputs=output, name=f"{model_name}_{activation}")
        y_pred_probs = model.predict(testX_preprocessed, verbose=0)

        # Use softmax to get predictions if activation is not softmax
        if activation == "softmax":
            predY = np.argmax(y_pred_probs, axis=1)
        else:
            predY = np.argmax(tf.nn.softmax(y_pred_probs, axis=1), axis=1)

        acc = accuracy_score(testY, predY) * 100
        results[model_name][activation] = acc
        print(f"Accuracy: {acc:.2f}%")

#======================= Summary Report ====================================================#
print("\n======================= Accuracy Summary =======================")
for model, acc_dict in results.items():
    print(f"{model:15}", end=" | ")
    for activation in activations:
        print(f"{activation}: {acc_dict[activation]:.2f}%", end=" | ")
    print()



================= VGG16 =================
--- Activation: relu
Accuracy: 2.75%
--- Activation: softmax
Accuracy: 4.45%
--- Activation: tanh
Accuracy: 7.20%
--- Activation: sigmoid
Accuracy: 6.70%

================= VGG19 =================
--- Activation: relu
Accuracy: 1.95%
--- Activation: softmax
Accuracy: 5.05%
--- Activation: tanh
Accuracy: 4.80%
--- Activation: sigmoid
Accuracy: 8.10%

================= ResNet50 =================
--- Activation: relu
Accuracy: 3.90%
--- Activation: softmax
Accuracy: 4.70%
--- Activation: tanh
Accuracy: 5.90%
--- Activation: sigmoid
Accuracy: 6.25%

================= InceptionV3 =================
--- Activation: relu
Accuracy: 5.15%
--- Activation: softmax
Accuracy: 5.80%
--- Activation: tanh
Accuracy: 6.75%
--- Activation: sigmoid
Accuracy: 5.80%

================= Xception =================
--- Activation: relu
Accuracy: 5.55%
--- Activation: softmax
Accuracy: 6.70%
--- Activation: tanh
Accuracy: 4.80%
--- Activation: sigmoid
Accuracy: 5.95%

==